<a href="https://colab.research.google.com/github/seunghuilee91/Deeplearning/blob/master/GAN_%2C_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
sys.path.append("/content/gdrive/My Drive/Colab Notebooks")


# GAN (Generative Adversarial Network, 2014)
---

- 생성 모델
- 기존의 어떤 방식보다 사실적인 결과를 만들어 내는 점에 엄청난 평가
- Discriminator(판별기)와 Generator(생성기)가 경쟁적으로 대립시켜(Adversarial) 학습
- 한번씩 교차로 학습
> GAN에서는 단순히 하나를 학습하기보다 경쟁시켜 학습하기 때문에 **판별기와 생성기가 함께 성장**. 
판별기는 이전의 분류모델로, 보통 맞다/아니다의 이진분류를 사용. 다른 모델과 차이점은 분류기의 입력에 생성 모델이 만들어낸 이미지를 사용하는 것임. 그러면, 생성 모델은 분류기가 분류할 이미지를 만들어내기 위해서 노력함. <br><br>
즉, **생성모델의 성능이 높아질 수록 분류모델은 어려운 문제를 풀게되고, 성능도 함께 오름**



## 모델 아키텍쳐

**[두가지 모델]**
- Generator Model : 생성한 데이터를 Discriminator가 진짜로 식하도록 하는 것이 목표

- Discriminator Model : Generator로 부터 전달된 데이터를 가짜로 인식하도록 하는 것이 목표


<BR>

<IMG SRC="https://pathmind.com/images/wiki/GANs.png">


**[동작 단계]**

1. Generator가 임의의 수를 입력받아 생성한 이미지로 변환한다.

2. 이렇게 생성된 이미지는 실제 데이터 세트에서 가져온 이미지들과 함께 Discriminator에게 전달된다.

3. Discrminator는 실제 이미지와 가짜 이미지를 판별하여 0과 1사이의 확률 값으로 반환한다. (1은 실제 이미지, 0은 가짜 이미지)

> 두 개의 모델을 경쟁을 시켜 최적의 결과 값을 만들어내는 것이다. 제로썸 게임처럼 서로 반대되는 목적함수 또는 손실함수를 통해 최적화하려고 시도 한다.







 ## 장점 및 한계점

**[장점]**
1. (압도적인) 생성 모델의 성능

2. 분류 모델의 효율적인 학습
 - 학습데이터가 적어도, 생성모델을 활용하여 보충할 수 있음
 -분류모델을 더 강인하게 학습할 수 있음

3. latent vector (잠재 벡터)
 - 생성모델의 결과를 다르게할 수 있는 잠재 공간이 있음




```
# 실제 있을 것 같은 것을 잘만들어낸다'라는 관점에서 보면 충분치 않은 데이터를 가지고 어느 수준의 데이터를 만들어 낼 수 있다.
# 보통 AI를 학습시킬 때는 pair의 입력 데이터가 필요한데 pair로 표현할 수 없는 경우가 있을때 GAN을 사용할 수 있다.(ex. 예술작품)
# (사용사례1) 가방 스케치만 가지고, 가방을 만들어낼 수 있다.
# (사용사례2) 불량 샘플이 필요한 경우 GAN 모델을 통해 몇 개 취득한 불량으로 부터 그 불량의 유형에 해당하는 다양한 케이스를 만들어낼 수 있다. 
# (사용사례3) 영어를 얘기하면 한국어로 만들어낸다.
```






   


**[한계점]**

**1. GAN은 결과가 불안정하다**

  - 기존 GAN만 가지고는 좋은 성능이 잘 안나왔다.
  - 수렴하지 못하는 경우가 생김
  - 생성기와 판별기가 고루 학습이 되어야하는데, 편향적으로 되어버릴 수 있음


**2. Black-box method**

Neural Network 자체의 한계라고 볼 수 있는데, 결정 변수나 주요 변수를 알 수 있는 다수의 머신러닝 기법들과 달리 Neural Network은 처음부터 끝까지 어떤 형태로 그러한 결과가 나오게 되었는지 그 과정을 알 수 없다.



**3. Generative Model 평가**
  
 GAN은 결과물 자체가 새롭게 만들어진 Sample 이다. 이를 기존 sample과 비교하여 얼마나 비슷한 지 확인할 수 있는 정량적 척도가 없고, 사람이 판단하더라도 이는 주관적 기준이기 때문에 얼마나 정확한지, 혹은 뛰어난지 판단하기 힘들다.

In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

In [ ]:
# Image processing
# transform = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
#                                      std=(0.5, 0.5, 0.5))])
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])])

# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [ ]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

# Device setting
D = D.to(device)
G = G.to(device)

# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [ ]:
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

In [ ]:

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()

In [ ]:
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))

In [ ]:
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

In [ ]:
# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

# DCGAN (Deep Convolutional GAN, 2015)

---

- GAN의 불안정성 등의 단점을 극복한 끝판왕, 이 논문 이후에 나오는 대다수의 GAN 구조는 전부 DCGAN의 구조를 사용했다고 해도 무방함 

- 의의 : 지도학습(Supervised Learning)에 Convolutional Neural Network (CNN)을 이용 (지도학습에서의 CNN의 성공과 비지도 학습 간의 격차를 줄이는 데에 큰 역할)

> DCGAN의 목표
1. Generator가 단순 기억으로 generate하지 않는다는 것을 보여줘야 한다. <BR> 2. z의 미세한 변동에 따른 generate결과가 연속적으로 부드럽게 이루어져야 한다(이를 walking in the latent space라고 한다).

## 모델 아키텍쳐

**기존 GAN Architecture** <BR>
기존 GAN은 자세히 살펴보면 다음과 같은 아주 간단하게 fully-connected로 연결되어 있다.
<IMG SRC="https://angrypark.github.io/images/2017-08-03-DCGAN-paper-reading/gan-architecture.png">

**CNN Architecture** <BR>
CNN은 이러한 fully-connected 구조 대신에 convolution, pooling, padding을 활용하여 레이어를 구성한다.

<IMG SRC="https://angrypark.github.io/images/2017-08-03-DCGAN-paper-reading/cnn-architecture.png">

<BR>
<BR>

---

###**DCGAN Architecture** <BR>
DCGAN은 결국, 기존 GAN에 존재했던 fully-connected구조의 대부분을 CNN 구조로 대체한 것인데, 앞서 언급했던 것처럼 엄청난 시도들 끝에 다음과 같이 구조를 결정하게 되었다.

> - Discriminator에서는 모든 pooling layers를 **strided convolutions** 로 바꾸고, Generator에서는 pooling layers를 **fractional-strided convolutions** 으로 바꾼다.
- Generator와 Discriminator에 batch-normalization을 사용한다. 논문에서는 이를 통해 deep generators의 초기 실패를 막는다고 하였다. 그러나 모든 layer에 다 적용하면 sample oscillation과 model instability의 문제가 발생하여 Generator output layer와 Discriminator input layer에는 적용하지 않았다고 한다.
- Fully-connected hidden layers를 삭제한다.
- Generator에서 모든 활성화 함수를 Relu를 쓰되, 마지막 결과에서만 Tanh를 사용한다.
- Discriminator에서는 모든 활성화 함수를 LeakyRelu를 쓴다.

<img src="https://2.bp.blogspot.com/-oMyhHfxOqiE/WKF4KlVYWJI/AAAAAAAABRs/6BDIypy1hn0U8MGRFxfVaXOcQDO7vX1cQCK4B/s1600/dcgan-architecture.PNG">



---
**Batch Normalization**<br>
 = 배치 정규화는 활성화함수의 활성화값 또는 출력값을 정규화(정규분포로 만든다)하는 작업

[장점]
- 학습 속도가 개선된다 (학습률을 높게 설정할 수 있기 때문)
- 가중치 초깃값 선택의 의존성이 적어진다 (학습을 할 때마다 출력값을 정규화하기 때문)
- 과적합(overfitting) 위험을 줄일 수 있다 (드롭아웃 같은 기법 대체 가능)
- Gradient Vanishing 문제 해결

<img src="http://sanghyukchun.github.io/images/post/88-5.png">

**Strided convolutions**

 ' convolutions는 필터를 거치며 크기가 작아진다
<img src="https://angrypark.github.io/images/2017-08-03-DCGAN-paper-reading/padding_strides.gif">



**Fractionally-strided convolutions**

'input에 padding을 하고 convolution을 하면서 오히려 크기가 더 커진다.
<img src="https://angrypark.github.io/images/2017-08-03-DCGAN-paper-reading/padding_strides_transposed.gif">


### **Visualization**
**Generated bedrooms**
<img src="https://4.bp.blogspot.com/-oLa4ynDytNc/WKLIUmTRIjI/AAAAAAAABSg/yK0Zyg7dFAIbjPJvIatke7ZKLUyOeIuBACK4B/s1600/dcgan-fig2.PNG">
▲ 한 번 epoch을 돌려 학습한 후 생성된 침실 사진

*앞서 얘기했던 1번 문제(Memorization)가 일어나지 않았다는 것을 검증하기 위해서 네트워크에 학습 데이터(training data)를 한 번만 보여준 결과*

<img src="https://angrypark.github.io/images/2017-08-03-DCGAN-paper-reading/visualization-1.png">

▲ 다섯 번 epoch을 돌려 학습한 후 생성된 침실 사진


**Walking in the latent space**<br>
 *DCGAN에서 학습된 Generator의 input인 z값(latent space)을 조금씩 바꿔가면서(walking) 생성되는 output 이미지가 어떻게 바뀌는지를 확인한 그림, 특히 중간에 여섯 번째 줄의 그림들을 보면 벽이었던 곳에서 서서히 커다란 창문이 있는 방으로 바뀌어가는 것을 볼 수 있음.*

<img src="https://angrypark.github.io/images/2017-08-03-DCGAN-paper-reading/visualization-2.png">

**Visualize filters (no longer black-box)**
<br>
*앞서 CNN이 black-box 모델이라 안이 어떻게 돌아가는지 모르고 쓸 뿐이라는 지적받았던 부분을 DCGAN 논문에서는 이 부분을 조금이나마 보여주고자 input까지 backpropagation을 하여 어떤 input이 discriminator가 학습한 feature의 어떤 부분을 active하게 하는지를 보여줌*

<img src="https://angrypark.github.io/images/2017-08-03-DCGAN-paper-reading/visualization-3.png">

**Applying arithmetic in the input space**
<img src="https://4.bp.blogspot.com/-0rilHKMWwTQ/WKFnwwv5QUI/AAAAAAAABRc/jDewod7JSikDgY5Xn957yPNzpCy4ZXmhwCK4B/w1200-h630-p-k-no-nu/dcgan-vector-arithmatic.PNG">


## 
[참고자료]

> 1시간만에 GAN(Generative Adversarial Network) 완전 정복하기
https://www.youtube.com/watch?v=odpjk7_tGY0

> Check out some other cool GAN projects
https://github.com/nashory/gans-awesome-applications
